In [4]:
from policies import Trajectory, p_s_from_rollouts, p_sa_from_rollouts, sr_from_rollouts, average_reward_from_rollouts
from plotting import plot_heatmap
import aggregation
import pickle

def read_pickle(save_name):
    """
    Thread-safe pickle read.
    Safe even if other threads are simultaneously calling dump_pickle()
    on the same file.
    """
    with open(save_name, "rb") as f:
        return pickle.load(f)

In [5]:

runs = read_pickle("out/mountaincar/runs_sa_eigenoptions_l1.pickle")



In [10]:
print(len(runs))
print(runs)

10
[[{'all_rollouts': [<policies.Trajectory object at 0x7fa3cc253a60>, <policies.Trajectory object at 0x7fa37318f1c0>, <policies.Trajectory object at 0x7fa37318ee00>, <policies.Trajectory object at 0x7fa37318edd0>, <policies.Trajectory object at 0x7fa37318e9e0>, <policies.Trajectory object at 0x7fa3731d4a00>, <policies.Trajectory object at 0x7fa373250dc0>, <policies.Trajectory object at 0x7fa3730319f0>, <policies.Trajectory object at 0x7fa3730ad780>, <policies.Trajectory object at 0x7fa3730e63b0>, <policies.Trajectory object at 0x7fa372f67040>, <policies.Trajectory object at 0x7fa372fa3b20>, <policies.Trajectory object at 0x7fa372e1c790>, <policies.Trajectory object at 0x7fa372e553c0>, <policies.Trajectory object at 0x7fa372e99ff0>, <policies.Trajectory object at 0x7fa372f12c80>, <policies.Trajectory object at 0x7fa372d53880>, <policies.Trajectory object at 0x7fa372dc44f0>, <policies.Trajectory object at 0x7fa372e09120>, <policies.Trajectory object at 0x7fa372c81db0>, <policies.Traject

TypeError: list indices must be integers or slices, not str